# Extracting wake and filtering it out of data

In [1]:
import scipy
from scipy import signal
from scipy import interpolate
from scipy import fftpack
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pandas as pd
import quantities as pq
import neo
from pathlib import Path
import xarray as xr
from IPython.display import display
from ipyfilechooser import FileChooser
import os
from ephyviewer import mkQApp, MainViewer, TraceViewer

In [2]:
dpath = "//10.69.168.1/crnldata/forgetting/Carla/"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = False, title = "<b>Open Ephys folder</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

FileChooser(path='\\10.69.168.1\crnldata\forgetting\Carla', filename='', title='<b>Open Ephys folder</b>', sho…

Stored 'dpath' (str)


In [5]:
folder_base = Path(dpath) 

DataRec2 = np.fromfile(folder_base, dtype="uint16")
All2 = DataRec2.reshape(-1,64)
All2.shape



PermissionError: [Errno 13] Permission denied: '\\\\10.69.168.1\\crnldata\\forgetting\\Carla\\OpenEphysData\\Vert0-31_Jaune32-64_2024-12-13_14-13-58_TestLFPs\\Record Node 103\\experiment1\\recording1\\continuous\\OE_FPGA_Acquisition_Board-101.Rhythm Data'

In [6]:
Channels = '//10.69.168.1/crnldata/forgetting/Carla/OpenEphysData/LFPChannels_perMice.xlsx' 
allchannels = pd.read_excel(Channels)

In [7]:
mice='Vert'
print(mice)

Vert


In [8]:
EMGch1=int(allchannels[mice][3])

EMG  =  All2[:, EMGch1]

NameError: name 'All2' is not defined

In [22]:
  # Filter parameter :
f_lowcut = 200.
f_hicut = 400.
N = 3
fs = 1000
nyq = 0.5 * fs
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction


In [23]:
# Filter creation :
b, a = signal.butter(N, Wn, 'band')
print(EMG.shape)
filt_EMG = signal.filtfilt(b, a, EMG,)


(5559227,)


In [24]:
# Parameter and computation of CWT
w = 4.
freq = np.linspace(200, 400, 50)
widths = w*fs / (2*freq*np.pi)
EMGcwt = signal.cwt(EMG, signal.morlet2, widths, w=w)

C:\Users\AudreyHay\AppData\Local\Temp\ipykernel_40936\1810749961.py:5: DeprecationWarning: scipy.signal.cwt is deprecated in SciPy 1.12 and will be removed
in SciPy 1.15. We recommend using PyWavelets instead.

  EMGcwt = signal.cwt(EMG, signal.morlet2, widths, w=w)


In [25]:
# Projection calculation
absEMGcwt = np.absolute(EMGcwt)
proj_EMGcwt = np.sum(absEMGcwt, axis = 0)/50
mproj_EMGcwt = np.mean(proj_EMGcwt)
sdproj_EMGcwt = np.std(proj_EMGcwt)

               
LowFactorSd= 3
HighFactorSd= 6


In [26]:
# Assigning values wake (1, 2) and sleep (0)
numpnts = EMG.size
EMGstatusRaw = np.zeros(numpnts)
for ind in range(numpnts):
        if proj_EMGcwt[ind]<(mproj_EMGcwt + LowFactorSd*sdproj_EMGcwt): 
                    EMGstatusRaw[ind] = 0
        elif proj_EMGcwt[ind]>(mproj_EMGcwt + HighFactorSd*sdproj_EMGcwt):
                    EMGstatusRaw[ind] = 2
        else:
                    EMGstatusRaw[ind] = 1

In [30]:
# Compter le nombre d'occurrences de chaque état
state_counts = np.bincount(EMGstatusRaw.astype(int))

# Afficher les résultats en tenant compte des états absents
print(f"Nombre d'épisodes d'état 0 (sommeil): {state_counts[0] if len(state_counts) > 0 else 0}")
print(f"Nombre d'épisodes d'état 1 (intermédiaire): {state_counts[1] if len(state_counts) > 1 else 0}")
print(f"Nombre d'épisodes d'état 2 (éveil): {state_counts[2] if len(state_counts) > 2 else 0}")


Nombre d'épisodes d'état 0 (sommeil): 5559227
Nombre d'épisodes d'état 1 (intermédiaire): 0
Nombre d'épisodes d'état 2 (éveil): 0


In [27]:
# Expanding borders for wake (1, 2) and sleep (0) to ±1 s around detected muscular activity
EMGstatusRaw2 = np.zeros(numpnts)
for ind in range(numpnts):
                if EMGstatusRaw[ind]>1:
                    EMGstatusRaw2[ind-1000:ind+1000] = 2
                elif EMGstatusRaw[ind]==1:
                    for ind2 in range(ind-1000, ind+1000):
                        if ind2==numpnts:
                            break
                        elif EMGstatusRaw2[ind2]<2:
                            EMGstatusRaw2[ind2] = 1


EMGStatusBoolLib = (EMGstatusRaw2>1)
EMGStatusBoolCons = (EMGstatusRaw2>0)

          

In [28]:
 # Save files
LFP = All2[:,:]
LFPwake0 = LFP.copy()
LFPwake0[EMGStatusBoolLib] = 0
filename = folder_base.parent/ f'LFPwake0'
np.save(filename, LFPwake0)

LFPwakeremoved = LFP.copy()
LFPwakeremoved = LFPwakeremoved[~EMGStatusBoolLib, :]
filename = folder_base.parent/ f'LFPwakeremoved'
np.save(filename, LFPwakeremoved)
data = {
    'EMGstatus': EMGstatusRaw2,
    'BooleanLiberal' : EMGStatusBoolLib,
    'BooleanConservative' : EMGStatusBoolCons
        }
WakeFrame = pd.DataFrame(data, columns=['EMGstatus', 'BooleanLiberal', 'BooleanConservative'])
print(f'EMGframeBoolean.pkl being saved' )
filename = folder_base.parent/ f'EMGframeBoolean.pkl'
WakeFrame.to_pickle(filename)

EMGframeBoolean.pkl being saved
